# Chapter 5

## Initial Setup

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%%time

# Load packages.
import sys
import torch  
import tiktoken

ancillar_path = "/llm_app/notebooks/build_large_language_models_from_scratch/"
if ancillar_path not in sys.path:
    sys.path.append(ancillar_path)

import ancillar as aux

CPU times: user 1.14 s, sys: 720 ms, total: 1.86 s
Wall time: 2.43 s


In [4]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size.
    "context_length": 256, # Shortened context length (orig: 1024).
    "emb_dim": 768,        # Embedding dimension.
    "n_heads": 12,         # Number of attention heads.
    "n_layers": 12,        # Number of layers.
    "drop_rate": 0.1,      # Dropout rate.
    "qkv_bias": False      # Query-key-value bias.
}

torch.manual_seed(123);

## Using GPT to Generate Text

In [5]:
model = aux.GPTModel(GPT_CONFIG_124M)

# Disable dropout during inference.
model.eval();  

In [6]:
# Listing 5.1: Utility functions for text to token ID conversion.

def text_to_token_ids(text, tokenizer):
    
    encoded = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
    
    # Add batch dimension.
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) 
    
    return encoded_tensor


def token_ids_to_text(token_ids, tokenizer):

    # Remove batch dimension.
    flat = token_ids.squeeze(0) 
    
    return tokenizer.decode(flat.tolist())

In [7]:
%%time

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = aux.generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

AttributeError: module 'torch' has no attribute 'inf'

In [8]:
from torch import inf

ImportError: cannot import name 'inf' from 'torch' (/usr/local/lib/python3.8/dist-packages/torch/__init__.py)